# Installing required libraries:

In [ ]:
pip install tkinter
pip install pillow


# Importing required libraries:

In [1]:
import tkinter as tk
from tkinter import *
import cv2
from PIL import Image, ImageTk
import numpy as np

# GUI(graphical user interface) to see the comparison:

In [3]:
cap1 = cv2.VideoCapture("C:\\Users\\DSK 8920444598\\Videos\\lane_detection\\test5.mp4")
cap2 = cv2.VideoCapture("C:\\Users\\DSK 8920444598\\Videos\\lane_detection\\test_5b.mp4")
root = tk.Tk()
root.title("Video Streams")
frame1 = Frame(root, borderwidth=2, relief="ridge", bg="green")
frame1.pack(side=LEFT, padx=10, pady=10)
frame2 = Frame(root, borderwidth=2, relief="ridge", bg="green")
frame2.pack(side=LEFT, padx=10, pady=10)
label1 = Label(frame1, text="Input Video", font=("Helvetica", 14))
label1.pack(padx=10, pady=10)
lmain = Label(frame1)
lmain.pack()
label2 = Label(frame2, text="Processed Video", font=("Helvetica", 14))
label2.pack(padx=10, pady=10)
lmain2 = Label(frame2)
lmain2.pack()
close_button = Button(root, text="Close App", command=root.quit, bg="red", fg="white", font=("Helvetica", 12))
close_button.pack(pady=10)
show_info = True
dark_mode = False
ym_per_pix = 30/720
xm_per_pix = 3.7/700
speed_factor = 1.0
toggle_info_button = Button(root, text="Toggle Information", command=lambda: toggle_info_display(), bg="blue", fg="white", font=("Helvetica", 15))
toggle_info_button.pack(pady=10)
speed_scale = Scale(root, from_=0.5, to=2.0, resolution=0.1, orient=HORIZONTAL, label="Speed", font=("Helvetica", 12))
speed_scale.pack(pady=10)
dark_mode_button = Button(root, text="Toggle Dark Mode", command=lambda: toggle_dark_mode(), bg="purple", fg="white", font=("Helvetica", 15))
dark_mode_button.pack(pady=10)
def update_speed(new_speed):
     global speed_factor
     speed_factor = new_speed
def calculate_curvature(ploty, left_fit, right_fit):
     y_eval = np.max(ploty)
    
     left_curvature = ((1 + (2*left_fit[0]*y_eval*ym_per_pix + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
     right_curvature = ((1 + (2*right_fit[0]*y_eval*ym_per_pix + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

     return left_curvature, right_curvature
def toggle_info_display():
     global show_info
     show_info = not show_info
def toggle_dark_mode():
     global dark_mode
     dark_mode = not dark_mode
    
     if dark_mode:
         root.configure(bg="black")
         frame1.configure(bg="black")
         frame2.configure(bg="black")
         label1.config(fg="black")
         label2.config(fg="black")
         close_button.config(bg="darkred")
         toggle_info_button.config(bg="darkblue")
         dark_mode_button.config(bg="darkgreen")
         speed_scale.config(bg="black", fg="white")
     else:
         root.configure(bg="white")
         frame1.configure(bg="green")
         frame2.configure(bg="green")
         label1.config(fg="black")
         label2.config(fg="black")
         close_button.config(bg="red")
         toggle_info_button.config(bg="blue")
         dark_mode_button.config(bg="purple")
         speed_scale.config(bg="white", fg="black")
def update_predicted_behavior(left_curvature, right_curvature):
     global predicted_behavior_text
    
     if abs(left_curvature) > 2000 or abs(right_curvature) > 2000:
         predicted_behavior_text.set("Sharp Turn Ahead")
     elif abs(left_curvature) > 1000 or abs(right_curvature) > 1000:
         predicted_behavior_text.set("Curve Ahead")
     else:
         predicted_behavior_text.set("Maintaining Course")
def simulate_lane_tracking(ploty, left_fit, right_fit, frame_shape):
     left_lane = np.column_stack((np.polyval(left_fit, ploty), ploty)).astype(np.int32)
     right_lane = np.column_stack((np.polyval(right_fit, ploty), ploty)).astype(np.int32)
    
     lane_img = np.zeros((frame_shape[0], frame_shape[1], 3), dtype=np.uint8)
     cv2.fillPoly(lane_img, [left_lane], (0, 255, 0)) # Left lane in green
     cv2.fillPoly(lane_img, [right_lane], (0, 255, 0)) # Right lane in green
    
     return lane_img
def show_vid():
     global cap1, lmain, show_info, speed_factor
     ret, frame = cap1.read()
     if ret:
         left_fit = np.polyfit(np.linspace(0, frame.shape[0] - 1, frame.shape[0]), np.random.rand(frame.shape[0]), 2)
         right_fit = np.polyfit(np.linspace(0, frame.shape[0] - 1, frame.shape[0]), np.random.rand(frame.shape[0]), 2)
        
         ploty = np.linspace(0, frame.shape[0] - 1, frame.shape[0])
         left_curvature, right_curvature = calculate_curvature(ploty, left_fit, right_fit)
    
         if show_info:
             cv2.putText(frame, f"Left Curvature: {left_curvature:.2f} m", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 1)
             cv2.putText(frame, f"Right Curvature: {right_curvature:.2f} m", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 1)
    
         simulated_lane_img = simulate_lane_tracking(ploty, left_fit, right_fit, frame.shape)
         combined_frame = cv2.addWeighted(frame, 1, simulated_lane_img, 0.3, 0)
        
         update_predicted_behavior(left_curvature, right_curvature)
        
         combined_frame = cv2.resize(combined_frame, (480, 360))
         pic = cv2.cvtColor(combined_frame, cv2.COLOR_BGR2RGB)
         img = Image.fromarray(pic)
         imgtk = ImageTk.PhotoImage(image=img)
         lmain.imgtk = imgtk
         lmain.configure(image=imgtk)
         delay = int(20 / speed_factor) if speed_factor > 0 else 1
         lmain.after(delay, show_vid)

def show_vid2():
     global cap2, lmain2
     ret, frame = cap2.read()
     if ret:
         frame = cv2.resize(frame, (480, 360))
         pic = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
         img = Image.fromarray(pic)
         imgtk = ImageTk.PhotoImage(image=img)
         lmain2.imgtk = imgtk
         lmain2.configure(image=imgtk)
         lmain2.after(20, show_vid2)

def on_speed_change(value):
     new_speed = float(value)
     update_speed(new_speed)

speed_scale.config(command=on_speed_change)
# Create a label for displaying predicted behavior
predicted_behavior_label = Label(root, text="Predicted Behavior:", font=("Helvetica", 14))
predicted_behavior_label.pack(pady=10)
predicted_behavior_text = StringVar()
predicted_behavior_text.set("Maintaining Course")
predicted_behavior_display = Label(root, textvariable=predicted_behavior_text, font=("Helvetica", 12))
predicted_behavior_display.pack(pady=10)
# Start the main loop
show_vid()
show_vid2()
root.mainloop()